In [1]:

include("../general_modules/channels.jl")
include("../general_modules/mesh.jl")
using .channels
using .mesh
using JSON

In [2]:
data = JSON.parsefile("test.json")
Systems_params= data["Systems"]
mesh_params= data["Mesh"]

α= α3b(0.5,0.5,1,Systems_params["lmax"],Systems_params["lmin"],Systems_params["λmax"],Systems_params["λmin"],Systems_params["s1"],Systems_params["s2"],Systems_params["s3"],Systems_params["t1"],Systems_params["t2"],Systems_params["t3"],Systems_params["MT"])

cosθi, dcosθi, xi, dxi, yi, dyi= initialmesh(mesh_params["nθ"],mesh_params["nx"],mesh_params["ny"],Float64(mesh_params["xmax"]),Float64(mesh_params["ymax"]),Float64(mesh_params["alpha"]))

For J=0.5 T=0.5 parity=1 Number of channels: 8


([-0.9931285991850949, -0.9639719272779138, -0.912234428251326, -0.8391169718222188, -0.7463319064601508, -0.636053680726515, -0.5108670019508271, -0.37370608871541955, -0.2277858511416451, -0.07652652113349732, 0.07652652113349732, 0.2277858511416451, 0.37370608871541955, 0.5108670019508271, 0.636053680726515, 0.7463319064601508, 0.8391169718222188, 0.912234428251326, 0.9639719272779138, 0.9931285991850949], [0.017614007139152055, 0.04060142980038697, 0.06267204833410904, 0.08327674157670457, 0.10193011981724034, 0.11819453196151841, 0.13168863844917655, 0.142096109318382, 0.14917298647260377, 0.15275338713072587, 0.15275338713072587, 0.14917298647260377, 0.142096109318382, 0.13168863844917655, 0.11819453196151841, 0.10193011981724034, 0.08327674157670457, 0.06267204833410904, 0.04060142980038697, 0.017614007139152055], [0.004358596129354452, 0.022968411603823145, 0.056462081708942215, 0.1048696848528175, 0.16822495102374077, 0.24657121555841388, 0.3399621299358039, 0.4484619677032939

In [27]:
for i in 1:mesh_params["nx"]
    println(xi[i], "   ", dxi[i])
end

0.004358596129354452   0.011185933400080516
0.022968411603823145   0.026046045706668262
0.056462081708942215   0.040945736465529466
0.1048696848528175   0.05587505159063575
0.16822495102374077   0.07084269503567375
0.24657121555841388   0.08585877130790606
0.3399621299358039   0.10093376523185128
0.4484619677032939   0.11607843357337054
0.5721458984302754   0.13130381662877735
0.7111002889020385   0.14662127574327033
0.865423046953378   0.16204253994203904
1.0352240148006955   0.1775797580586774
1.2206254169517339   0.19324555576379496
1.4217623676962623   0.20905309778320763
1.6387834436668216   0.22501615594576957
1.871851327707601   0.24114918390746576
2.121143531223285   0.257467399571867
2.3868532032890375   0.27398687640842606
2.6691900361049306   0.2907246450768459
2.968381277905127   0.3076988070104802
3.28467286622724   0.32492866190527825
3.618330696566028   0.3424348514166627
3.9696420439471383   0.3602395217995102
4.338917157948008   0.3783665087568519
4.726491055273016   0

In [8]:
using Revise
Revise.revise()
includet("Gcoefficient.jl")
using .Gcoefficient

initialY(λmax, lmax, nθ, nx, ny, cosθi, xi, yi, P::Char)
return Yλαout, Ylαin, Yλαin

Yλin[iθ, iy, ix, nch, perm_index]

In [9]:
Yλout, Ylin, Yλin = initialY(Systems_params["λmax"], Systems_params["lmax"], mesh_params["nθ"],mesh_params["nx"],mesh_params["ny"], cosθi, xi, yi)

(ComplexF64[0.2820947917738782 + 0.0im 0.04043253758840054 - 0.0im -0.4852451282044655 + 0.0im -0.04043253758840054 - 0.0im; 0.2820947917738782 + 0.0im 0.09190282275132813 - 0.0im -0.4709991050718876 + 0.0im -0.09190282275132813 - 0.0im; … ; 0.2820947917738782 + 0.0im 0.09190282275132805 - 0.0im 0.4709991050718876 + 0.0im -0.09190282275132805 - 0.0im; 0.2820947917738782 + 0.0im 0.04043253758840047 - 0.0im 0.4852451282044655 + 0.0im -0.04043253758840047 - 0.0im], ComplexF64[0.2820947917738782 + 0.0im 0.2820947917738782 + 0.0im … 0.2820947917738782 + 0.0im 0.2820947917738782 + 0.0im; 0.2820947917738782 + 0.0im 0.2820947917738782 + 0.0im … 0.2820947917738782 + 0.0im 0.2820947917738782 + 0.0im; … ; 0.2820947917738782 + 0.0im 0.2820947917738782 + 0.0im … 0.2820947917738782 + 0.0im 0.2820947917738782 + 0.0im; 0.2820947917738782 + 0.0im 0.2820947917738782 + 0.0im … 0.2820947917738782 + 0.0im 0.2820947917738782 + 0.0im;;; 0.2820947917738782 + 0.0im 0.2820947917738782 + 0.0im … 0.28209479177387

In [10]:
println(Yλin[5, 1, 1, 1, 1])
println(Yλin[5, 1, 1, 4, 1])

0.2820947917738782 + 0.0im
0.2286970261016521 - 2.80073080989907e-17im


YYcoupling(α, nθ, ny, nx, Ylin, Yλin, Yλout)
    return Y4[iθ, iy, ix, αin, αout, 2]

In [11]:
Y4 = YYcoupling(α, mesh_params["nθ"],mesh_params["ny"],mesh_params["nx"], Ylin, Yλin, Yλout)

20×60×60×8×8×2 Array{ComplexF64, 6}:
[:, :, 1, 1, 1, 1] =
 0.5+0.0im  0.5+0.0im  0.5+0.0im  …  0.5+0.0im  0.5+0.0im  0.5+0.0im
 0.5+0.0im  0.5+0.0im  0.5+0.0im     0.5+0.0im  0.5+0.0im  0.5+0.0im
 0.5+0.0im  0.5+0.0im  0.5+0.0im     0.5+0.0im  0.5+0.0im  0.5+0.0im
 0.5+0.0im  0.5+0.0im  0.5+0.0im     0.5+0.0im  0.5+0.0im  0.5+0.0im
 0.5+0.0im  0.5+0.0im  0.5+0.0im     0.5+0.0im  0.5+0.0im  0.5+0.0im
 0.5+0.0im  0.5+0.0im  0.5+0.0im  …  0.5+0.0im  0.5+0.0im  0.5+0.0im
 0.5+0.0im  0.5+0.0im  0.5+0.0im     0.5+0.0im  0.5+0.0im  0.5+0.0im
 0.5+0.0im  0.5+0.0im  0.5+0.0im     0.5+0.0im  0.5+0.0im  0.5+0.0im
 0.5+0.0im  0.5+0.0im  0.5+0.0im     0.5+0.0im  0.5+0.0im  0.5+0.0im
 0.5+0.0im  0.5+0.0im  0.5+0.0im     0.5+0.0im  0.5+0.0im  0.5+0.0im
 0.5+0.0im  0.5+0.0im  0.5+0.0im  …  0.5+0.0im  0.5+0.0im  0.5+0.0im
 0.5+0.0im  0.5+0.0im  0.5+0.0im     0.5+0.0im  0.5+0.0im  0.5+0.0im
 0.5+0.0im  0.5+0.0im  0.5+0.0im     0.5+0.0im  0.5+0.0im  0.5+0.0im
 0.5+0.0im  0.5+0.0im  0.5+0.0im     0.5+0.0i

In [ ]:
G= Gαα( mesh_params["nθ"], mesh_params["ny"],mesh_params["nx"], α, Systems_params["s1"],Systems_params["s2"],Systems_params["s3"],Systems_params["t1"],Systems_params["t2"],Systems_params["t3"],Y4)

20×60×60×8×8×2 Array{ComplexF64, 6}:
[:, :, 1, 1, 1, 1] =
 0.125+0.0im  0.125+0.0im  0.125+0.0im  …  0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im  …  0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im  …  0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.

In [35]:
G1=computeGcoffecient(α,Systems_params["lmax"],Systems_params["λmax"],Systems_params["s1"],Systems_params["s2"],Systems_params["s3"],Systems_params["t1"],Systems_params["t2"],Systems_params["t3"], mesh_params["nθ"], mesh_params["nx"],mesh_params["ny"], cosθi, xi, yi)

20×60×60×8×8×2 Array{ComplexF64, 6}:
[:, :, 1, 1, 1, 1] =
 0.125+0.0im  0.125+0.0im  0.125+0.0im  …  0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im  …  0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im  …  0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.125+0.0im
 0.125+0.0im  0.125+0.0im  0.125+0.0im     0.125+0.0im  0.